# Soft Actor-Critic (SAC): Maximum Entropy RL

SAC is a state-of-the-art algorithm for continuous control that learns to be optimally random!

## What You'll Learn

By the end of this notebook, you'll understand:
- The jazz musician analogy: why controlled randomness helps
- Maximum entropy RL: reward + entropy
- SAC's key innovations (twin critics, automatic temperature)
- Off-policy learning: why SAC is sample efficient
- Using SAC with Stable-Baselines3
- When to choose SAC vs PPO

**Prerequisites:** Notebook 2 (PPO From Scratch)

**Time:** ~30 minutes

---
## The Big Picture: The Jazz Musician Analogy

```
    ┌────────────────────────────────────────────────────────────────┐
    │          THE JAZZ MUSICIAN ANALOGY                             │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  Imagine two musicians learning to improvise...               │
    │                                                                │
    │  CLASSICAL MUSICIAN (Standard RL):                            │
    │    "I'll learn THE BEST note for each moment"                │
    │    → Always plays the same lick in similar situations        │
    │    → Predictable, gets stuck in routines                     │
    │    → If one approach fails, struggles to adapt               │
    │                                                                │
    │  JAZZ MUSICIAN (SAC - Maximum Entropy):                       │
    │    "I'll learn MANY good options and stay creative!"         │
    │    → Keeps multiple approaches ready                         │
    │    → Random but controlled improvisation                     │
    │    → Adapts easily when things change                        │
    │                                                                │
    │  SAC'S INSIGHT:                                               │
    │    Don't just maximize reward - also maximize ENTROPY!       │
    │    Entropy = "controlled randomness" = keeping options open  │
    │                                                                │
    │    J(π) = E[Σ r(s,a)] + α × H(π)                             │
    │            └─rewards─┘   └─entropy bonus─┘                   │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch, Circle, Wedge

try:
    import gymnasium as gym
except ImportError:
    import gym

# Check if Stable-Baselines3 is available
try:
    from stable_baselines3 import SAC, PPO
    from stable_baselines3.common.env_util import make_vec_env
    from stable_baselines3.common.evaluation import evaluate_policy
    import torch
    SB3_AVAILABLE = True
    print("✓ Stable-Baselines3 is installed!")
except ImportError:
    SB3_AVAILABLE = False
    print("✗ Stable-Baselines3 not installed.")
    print("  Install with: pip install stable-baselines3[extra]")

In [ ]:
# Visualize standard RL vs Maximum Entropy RL

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Left: Standard RL (deterministic policy)
ax1 = axes[0]
ax1.set_xlim(-3, 3)
ax1.set_ylim(-3, 3)

# Show deterministic policy
ax1.scatter([0], [0], s=200, c='blue', zorder=5, label='Current state')
ax1.arrow(0, 0, 1.5, 1, head_width=0.2, head_length=0.1, fc='red', ec='red', linewidth=3)
ax1.text(1.8, 1.2, 'Always this\naction!', fontsize=10, color='red')

ax1.set_xlabel('Action dimension 1', fontsize=11)
ax1.set_ylabel('Action dimension 2', fontsize=11)
ax1.set_title('Standard RL: Deterministic\n"Always pick THE BEST action"', fontsize=12, fontweight='bold', color='#d32f2f')
ax1.grid(True, alpha=0.3)
ax1.set_aspect('equal')

# Right: SAC (stochastic policy with high entropy)
ax2 = axes[1]
ax2.set_xlim(-3, 3)
ax2.set_ylim(-3, 3)

# Show stochastic policy as distribution
ax2.scatter([0], [0], s=200, c='blue', zorder=5, label='Current state')

# Draw distribution cloud
theta = np.linspace(0, 2*np.pi, 100)
for r in [0.5, 1.0, 1.5]:
    alpha = 0.3 - r/5
    x = 1 + r * np.cos(theta) * 0.8
    y = 0.5 + r * np.sin(theta) * 0.8
    ax2.fill(x, y, alpha=alpha, color='green')

# Sample actions
np.random.seed(42)
sample_x = 1 + np.random.randn(20) * 0.5
sample_y = 0.5 + np.random.randn(20) * 0.5
ax2.scatter(sample_x, sample_y, s=30, c='green', alpha=0.7, zorder=4)

ax2.arrow(0, 0, 0.8, 0.4, head_width=0.15, head_length=0.1, fc='green', ec='green', linewidth=2, alpha=0.7)
ax2.text(2.2, 1.2, 'Distribution of\ngood actions!', fontsize=10, color='green')

ax2.set_xlabel('Action dimension 1', fontsize=11)
ax2.set_ylabel('Action dimension 2', fontsize=11)
ax2.set_title('SAC: Maximum Entropy\n"Many good actions, stay creative!"', fontsize=12, fontweight='bold', color='#388e3c')
ax2.grid(True, alpha=0.3)
ax2.set_aspect('equal')

plt.tight_layout()
plt.show()

print("\nMAXIMUM ENTROPY BENEFITS:")
print("  • Better exploration (tries more actions)")
print("  • More robust (multiple solutions ready)")
print("  • Easier to fine-tune later")
print("  • Works better in stochastic environments")

---
## Maximum Entropy RL: The Core Idea

```
    ┌────────────────────────────────────────────────────────────────┐
    │              MAXIMUM ENTROPY OBJECTIVE                         │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  STANDARD RL OBJECTIVE:                                       │
    │    J(π) = E[Σ γᵗ r(sₜ, aₜ)]                                  │
    │    "Maximize expected discounted rewards"                    │
    │                                                                │
    │  MAXIMUM ENTROPY OBJECTIVE:                                   │
    │    J(π) = E[Σ γᵗ (r(sₜ, aₜ) + α H(π(·|sₜ)))]                │
    │                               └───────────┘                   │
    │                               Entropy bonus!                  │
    │                                                                │
    │  WHAT IS ENTROPY?                                             │
    │    H(π) = -E[log π(a|s)]                                     │
    │    • High entropy: Actions spread out (random)               │
    │    • Low entropy: Actions concentrated (deterministic)       │
    │                                                                │
    │  THE α (TEMPERATURE) PARAMETER:                               │
    │    • α high: Prioritize entropy (more exploration)           │
    │    • α low: Prioritize reward (more exploitation)            │
    │    • SAC learns α automatically!                             │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
def compute_entropy(probs):
    """Compute entropy of a probability distribution."""
    probs = np.array(probs) + 1e-10
    return -np.sum(probs * np.log(probs))

# Demonstrate entropy
print("ENTROPY: MEASURING RANDOMNESS")
print("="*60)

distributions = [
    ([1.0, 0.0, 0.0, 0.0], "Deterministic (always action 1)"),
    ([0.7, 0.2, 0.1, 0.0], "Low entropy (mostly action 1)"),
    ([0.4, 0.3, 0.2, 0.1], "Medium entropy (preferences)"),
    ([0.25, 0.25, 0.25, 0.25], "Maximum entropy (uniform)"),
]

print("\nAction probability distributions:")
entropies = []
for probs, desc in distributions:
    H = compute_entropy(probs)
    entropies.append(H)
    print(f"  {probs} → H = {H:.3f} ({desc})")

print("\n" + "="*60)
print("Higher entropy = more randomness = more exploration!")
print("="*60)

In [ ]:
# Visualize entropy and its effect

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Left: Different entropy levels
ax1 = axes[0]
actions = ['A1', 'A2', 'A3', 'A4']
x = np.arange(len(actions))
width = 0.2

colors = ['#d32f2f', '#ff9800', '#4caf50', '#2196f3']
for i, (probs, desc) in enumerate(distributions):
    ax1.bar(x + i*width, probs, width, label=f'H={entropies[i]:.2f}', color=colors[i], alpha=0.8)

ax1.set_xticks(x + 1.5*width)
ax1.set_xticklabels(actions)
ax1.set_ylabel('Probability', fontsize=11)
ax1.set_title('Action Distributions with Different Entropy', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

# Middle: Entropy vs spread
ax2 = axes[1]
# Continuous case: Gaussian with different std
x_vals = np.linspace(-4, 4, 100)
stds = [0.5, 1.0, 2.0]
colors = ['#d32f2f', '#4caf50', '#2196f3']

for std, color in zip(stds, colors):
    y = np.exp(-x_vals**2 / (2*std**2)) / (std * np.sqrt(2*np.pi))
    # Entropy of Gaussian: 0.5 * ln(2πe σ²)
    H_gaussian = 0.5 * np.log(2 * np.pi * np.e * std**2)
    ax2.plot(x_vals, y, color=color, linewidth=2, label=f'σ={std}, H={H_gaussian:.2f}')
    ax2.fill_between(x_vals, y, alpha=0.2, color=color)

ax2.set_xlabel('Action value', fontsize=11)
ax2.set_ylabel('Probability density', fontsize=11)
ax2.set_title('Continuous Actions: Gaussian Policies', fontsize=12, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Right: Effect of temperature α
ax3 = axes[2]
alphas = np.linspace(0, 2, 100)
exploration = 1 - np.exp(-alphas*2)
exploitation = np.exp(-alphas)

ax3.fill_between(alphas, 0, exploration, alpha=0.3, color='#2196f3', label='Exploration')
ax3.fill_between(alphas, 0, exploitation, alpha=0.3, color='#4caf50', label='Exploitation')
ax3.plot(alphas, exploration, 'b-', linewidth=2)
ax3.plot(alphas, exploitation, 'g-', linewidth=2)

ax3.axvline(x=0.5, color='red', linestyle='--', label='Typical α')
ax3.set_xlabel('Temperature α', fontsize=11)
ax3.set_ylabel('Behavior emphasis', fontsize=11)
ax3.set_title('Temperature Controls Exploration', fontsize=12, fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## SAC Architecture: The Key Components

```
    ┌────────────────────────────────────────────────────────────────┐
    │              SAC ARCHITECTURE                                  │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  1. ACTOR (Policy Network):                                   │
    │     • Outputs: μ(s), σ(s) for Gaussian                       │
    │     • Sample: a = μ + σ × ε, where ε ~ N(0,1)                │
    │     • Uses reparameterization trick                          │
    │                                                                │
    │  2. TWIN CRITICS (Two Q-Networks):                           │
    │     • Q₁(s, a) and Q₂(s, a)                                  │
    │     • Take minimum: min(Q₁, Q₂)                              │
    │     • Prevents overestimation (like TD3)                     │
    │                                                                │
    │  3. TARGET NETWORKS:                                          │
    │     • Soft update: θ' ← τθ + (1-τ)θ'                        │
    │     • Stabilizes training                                    │
    │                                                                │
    │  4. AUTOMATIC TEMPERATURE (α):                                │
    │     • Learned to maintain target entropy                     │
    │     • No manual tuning needed!                               │
    │                                                                │
    │  5. REPLAY BUFFER (Off-Policy):                               │
    │     • Store and reuse past experiences                       │
    │     • Much more sample efficient than PPO                    │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Visualize SAC architecture

fig, ax = plt.subplots(figsize=(14, 10))
ax.set_xlim(0, 14)
ax.set_ylim(0, 12)
ax.axis('off')
ax.set_title('SAC Architecture: Actor-Critic with Maximum Entropy', fontsize=16, fontweight='bold')

# State input
state_box = FancyBboxPatch((6, 10), 2, 1, boxstyle="round,pad=0.1",
                            facecolor='#e3f2fd', edgecolor='#1976d2', linewidth=3)
ax.add_patch(state_box)
ax.text(7, 10.5, 'State s', ha='center', fontsize=11, fontweight='bold')

# Actor (Policy)
actor_box = FancyBboxPatch((1, 6.5), 3.5, 2.5, boxstyle="round,pad=0.1",
                            facecolor='#c8e6c9', edgecolor='#388e3c', linewidth=3)
ax.add_patch(actor_box)
ax.text(2.75, 8.3, 'ACTOR', ha='center', fontsize=11, fontweight='bold', color='#388e3c')
ax.text(2.75, 7.6, 'π(a|s)', ha='center', fontsize=10)
ax.text(2.75, 7, 'Outputs: μ, σ', ha='center', fontsize=9)

# Twin Critics
critic1_box = FancyBboxPatch((5.5, 6.5), 1.8, 2.5, boxstyle="round,pad=0.1",
                              facecolor='#e1bee7', edgecolor='#7b1fa2', linewidth=2)
ax.add_patch(critic1_box)
ax.text(6.4, 8.2, 'Q₁', ha='center', fontsize=11, fontweight='bold', color='#7b1fa2')
ax.text(6.4, 7.5, 'Q₁(s,a)', ha='center', fontsize=9)

critic2_box = FancyBboxPatch((7.7, 6.5), 1.8, 2.5, boxstyle="round,pad=0.1",
                              facecolor='#e1bee7', edgecolor='#7b1fa2', linewidth=2)
ax.add_patch(critic2_box)
ax.text(8.6, 8.2, 'Q₂', ha='center', fontsize=11, fontweight='bold', color='#7b1fa2')
ax.text(8.6, 7.5, 'Q₂(s,a)', ha='center', fontsize=9)

# Min box
min_box = FancyBboxPatch((6.4, 4.5), 2.2, 1.2, boxstyle="round,pad=0.1",
                          facecolor='#fff3e0', edgecolor='#f57c00', linewidth=2)
ax.add_patch(min_box)
ax.text(7.5, 5.1, 'min(Q₁, Q₂)', ha='center', fontsize=10, fontweight='bold')

# Temperature α
temp_box = FancyBboxPatch((10.5, 6.5), 2.5, 2.5, boxstyle="round,pad=0.1",
                           facecolor='#ffcdd2', edgecolor='#d32f2f', linewidth=2)
ax.add_patch(temp_box)
ax.text(11.75, 8.2, 'Temperature', ha='center', fontsize=10, fontweight='bold', color='#d32f2f')
ax.text(11.75, 7.5, 'α (learned)', ha='center', fontsize=10)
ax.text(11.75, 6.9, 'Auto-tuned!', ha='center', fontsize=9, style='italic')

# Replay Buffer
buffer_box = FancyBboxPatch((1, 2.5), 4, 2, boxstyle="round,pad=0.1",
                             facecolor='#bbdefb', edgecolor='#1976d2', linewidth=2)
ax.add_patch(buffer_box)
ax.text(3, 4, 'Replay Buffer', ha='center', fontsize=11, fontweight='bold', color='#1976d2')
ax.text(3, 3.2, 'Stores (s, a, r, s\')', ha='center', fontsize=9)

# Arrows
ax.annotate('', xy=(2.75, 9), xytext=(6, 10),
            arrowprops=dict(arrowstyle='->', lw=2, color='#666'))
ax.annotate('', xy=(7.5, 9), xytext=(7, 9.9),
            arrowprops=dict(arrowstyle='->', lw=2, color='#666'))
ax.annotate('', xy=(6.9, 6.4), xytext=(6.9, 5.8),
            arrowprops=dict(arrowstyle='->', lw=1, color='#666'))
ax.annotate('', xy=(8.1, 6.4), xytext=(8.1, 5.8),
            arrowprops=dict(arrowstyle='->', lw=1, color='#666'))

# Labels
ax.text(7.5, 1.5, 'OFF-POLICY: Sample from buffer, not current policy!', 
        ha='center', fontsize=11, fontweight='bold', color='#1976d2')
ax.text(7.5, 0.8, 'This makes SAC much more sample efficient than PPO.', 
        ha='center', fontsize=10)

plt.tight_layout()
plt.show()

print("\nSAC'S KEY INNOVATIONS:")
print("  1. Twin Critics: Take minimum to prevent overestimation")
print("  2. Automatic Temperature: No manual α tuning")
print("  3. Off-Policy: Reuse past experiences (sample efficient!)")
print("  4. Entropy Bonus: Encourages exploration")

---
## On-Policy vs Off-Policy: Why SAC is Sample Efficient

```
    ┌────────────────────────────────────────────────────────────────┐
    │              ON-POLICY vs OFF-POLICY                           │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  ON-POLICY (PPO, A2C):                                        │
    │    • Collect data with current policy                        │
    │    • Use data ONCE for update                                │
    │    • Throw away data after update                            │
    │    • Need lots of new data constantly                        │
    │                                                                │
    │  OFF-POLICY (SAC, TD3, DQN):                                  │
    │    • Store ALL past experiences in buffer                    │
    │    • Sample random batches for training                      │
    │    • Reuse data many times                                   │
    │    • Much more sample efficient!                             │
    │                                                                │
    │  ANALOGY:                                                     │
    │    On-policy: Student who only learns from today's class    │
    │    Off-policy: Student who reviews all past notes + today   │
    │                                                                │
    │  SAMPLE EFFICIENCY:                                           │
    │    SAC can learn from 100K samples what PPO needs 1M for!   │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Visualize on-policy vs off-policy

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Left: On-policy data flow
ax1 = axes[0]
ax1.set_xlim(0, 10)
ax1.set_ylim(0, 10)
ax1.axis('off')
ax1.set_title('ON-POLICY (PPO)\n"Use once, throw away"', fontsize=14, fontweight='bold', color='#d32f2f')

# Show data flow
for i in range(3):
    y = 7.5 - i * 2.5
    # Collect
    collect_box = FancyBboxPatch((1, y), 2, 1.5, boxstyle="round,pad=0.1",
                                  facecolor='#bbdefb', edgecolor='#1976d2', linewidth=2)
    ax1.add_patch(collect_box)
    ax1.text(2, y + 0.75, f'Batch {i+1}', ha='center', fontsize=10)
    
    # Arrow to update
    ax1.annotate('', xy=(4, y + 0.75), xytext=(3.2, y + 0.75),
                 arrowprops=dict(arrowstyle='->', lw=2, color='#388e3c'))
    
    # Update
    update_box = FancyBboxPatch((4.2, y), 2, 1.5, boxstyle="round,pad=0.1",
                                 facecolor='#c8e6c9', edgecolor='#388e3c', linewidth=2)
    ax1.add_patch(update_box)
    ax1.text(5.2, y + 0.75, 'Update', ha='center', fontsize=10)
    
    # Arrow to trash
    ax1.annotate('', xy=(7.5, y + 0.75), xytext=(6.4, y + 0.75),
                 arrowprops=dict(arrowstyle='->', lw=2, color='#d32f2f'))
    
    # Trash
    ax1.text(8, y + 0.75, '🗑️', ha='center', fontsize=20)

ax1.text(5, 1, '❌ Data used once, then discarded', ha='center', fontsize=10, color='#d32f2f')

# Right: Off-policy data flow
ax2 = axes[1]
ax2.set_xlim(0, 10)
ax2.set_ylim(0, 10)
ax2.axis('off')
ax2.set_title('OFF-POLICY (SAC)\n"Store and reuse forever"', fontsize=14, fontweight='bold', color='#388e3c')

# Replay buffer (big)
buffer_box = FancyBboxPatch((1, 3), 4, 5, boxstyle="round,pad=0.1",
                             facecolor='#bbdefb', edgecolor='#1976d2', linewidth=3)
ax2.add_patch(buffer_box)
ax2.text(3, 7.5, 'Replay Buffer', ha='center', fontsize=11, fontweight='bold')

# Show accumulated data
for i in range(5):
    y = 3.5 + i * 0.8
    mini_box = FancyBboxPatch((1.5, y), 3, 0.6, boxstyle="round,pad=0.05",
                               facecolor='#90caf9', edgecolor='#1976d2', linewidth=1)
    ax2.add_patch(mini_box)
    ax2.text(3, y + 0.3, f'Data {i+1}', ha='center', fontsize=8)

# Arrow: new data in
ax2.annotate('', xy=(3, 8.2), xytext=(3, 9),
             arrowprops=dict(arrowstyle='->', lw=2, color='#1976d2'))
ax2.text(3, 9.3, 'New data', ha='center', fontsize=9)

# Arrow: sample out
ax2.annotate('', xy=(6, 5.5), xytext=(5.2, 5.5),
             arrowprops=dict(arrowstyle='->', lw=2, color='#388e3c'))

# Update box
update_box = FancyBboxPatch((6.2, 4.5), 2.5, 2, boxstyle="round,pad=0.1",
                             facecolor='#c8e6c9', edgecolor='#388e3c', linewidth=2)
ax2.add_patch(update_box)
ax2.text(7.45, 5.8, 'Update', ha='center', fontsize=11, fontweight='bold')
ax2.text(7.45, 5, '(random batch)', ha='center', fontsize=9)

ax2.text(5, 1.5, '✓ Data reused many times!', ha='center', fontsize=10, color='#388e3c')
ax2.text(5, 0.8, '✓ 10x more sample efficient', ha='center', fontsize=10, color='#388e3c')

plt.tight_layout()
plt.show()

---
## Using SAC with Stable-Baselines3

In [ ]:
if SB3_AVAILABLE:
    print("SAC WITH STABLE-BASELINES3")
    print("="*60)
    
    # SAC works with CONTINUOUS action spaces
    # Pendulum-v1 is a classic continuous control task
    env = gym.make('Pendulum-v1')
    
    print(f"\nEnvironment: Pendulum-v1")
    print(f"  Observation space: {env.observation_space}")
    print(f"  Action space: {env.action_space}")
    print(f"  → Continuous action! (torque from -2 to +2)")
    
    # Create SAC model
    model = SAC(
        policy='MlpPolicy',
        env=env,
        learning_rate=3e-4,
        buffer_size=50000,        # Replay buffer size
        learning_starts=1000,      # Start training after this many steps
        batch_size=256,            # Batch size for training
        tau=0.005,                 # Soft update coefficient
        gamma=0.99,                # Discount factor
        train_freq=1,              # Update every step
        gradient_steps=1,          # Gradient steps per update
        ent_coef='auto',           # Automatic temperature!
        verbose=0,
    )
    
    print("\nCreated SAC model with:")
    print(f"  buffer_size: {model.buffer_size}")
    print(f"  batch_size: {model.batch_size}")
    print(f"  ent_coef: {model.ent_coef} (auto-tuned!)")
    
    # Train
    print("\nTraining for 10,000 timesteps...")
    model.learn(total_timesteps=10_000, progress_bar=True)
    
    # Evaluate
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
    print(f"\nEvaluation: {mean_reward:.1f} ± {std_reward:.1f}")
    print("(Pendulum: higher is better, max around -200)")
    
    env.close()
    print("\n" + "="*60)
else:
    print("Install Stable-Baselines3 to run this example.")

In [ ]:
if SB3_AVAILABLE:
    # Train longer and visualize
    print("TRAINING SAC FOR LONGER")
    print("="*60)
    
    env = gym.make('Pendulum-v1')
    model = SAC('MlpPolicy', env, verbose=0, ent_coef='auto')
    
    # Track rewards during training
    rewards_during_training = []
    
    print("Training with periodic evaluation...")
    for i in range(10):
        model.learn(total_timesteps=5000, reset_num_timesteps=False)
        mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=5, deterministic=True)
        rewards_during_training.append(mean_reward)
        print(f"  Step {(i+1)*5000:5d}: Mean reward = {mean_reward:.1f}")
    
    # Visualize
    fig, ax = plt.subplots(figsize=(10, 5))
    steps = [5000 * (i+1) for i in range(len(rewards_during_training))]
    ax.plot(steps, rewards_during_training, 'go-', linewidth=2, markersize=8)
    ax.axhline(y=-200, color='red', linestyle='--', label='Good performance')
    ax.set_xlabel('Training Steps', fontsize=11)
    ax.set_ylabel('Mean Reward', fontsize=11)
    ax.set_title('SAC Training on Pendulum-v1', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    env.close()
else:
    print("Install Stable-Baselines3 to run this example.")

---
## PPO vs SAC: When to Use Which?

```
    ┌────────────────────────────────────────────────────────────────┐
    │              PPO vs SAC COMPARISON                             │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  USE PPO WHEN:                                                │
    │    ✓ Discrete actions (games, text)                          │
    │    ✓ Simulations are cheap (can get lots of data)            │
    │    ✓ Need stable, reliable training                          │
    │    ✓ RLHF for language models                                │
    │                                                                │
    │  USE SAC WHEN:                                                │
    │    ✓ Continuous actions (robotics, control)                  │
    │    ✓ Real-world where data is expensive                      │
    │    ✓ Need sample efficiency                                  │
    │    ✓ Exploration is important                                │
    │                                                                │
    │  KEY DIFFERENCES:                                             │
    │    • PPO: On-policy, stable, less sample efficient           │
    │    • SAC: Off-policy, sample efficient, continuous only      │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Visualize PPO vs SAC comparison

fig, ax = plt.subplots(figsize=(12, 8))
ax.set_xlim(0, 12)
ax.set_ylim(0, 10)
ax.axis('off')
ax.set_title('PPO vs SAC: Choose the Right Tool', fontsize=16, fontweight='bold')

# PPO column
ppo_box = FancyBboxPatch((0.5, 2), 5, 6.5, boxstyle="round,pad=0.1",
                          facecolor='#e3f2fd', edgecolor='#1976d2', linewidth=3)
ax.add_patch(ppo_box)
ax.text(3, 8, 'PPO', ha='center', fontsize=14, fontweight='bold', color='#1976d2')

ppo_features = [
    '✓ Discrete & Continuous',
    '✓ Very Stable',
    '✓ Simple to tune',
    '✓ RLHF standard',
    '✗ Less sample efficient',
    '✗ Needs lots of data',
]
for i, feat in enumerate(ppo_features):
    color = '#388e3c' if feat.startswith('✓') else '#d32f2f'
    ax.text(1, 7 - i*0.7, feat, fontsize=10, color=color)

# SAC column
sac_box = FancyBboxPatch((6.5, 2), 5, 6.5, boxstyle="round,pad=0.1",
                          facecolor='#e8f5e9', edgecolor='#388e3c', linewidth=3)
ax.add_patch(sac_box)
ax.text(9, 8, 'SAC', ha='center', fontsize=14, fontweight='bold', color='#388e3c')

sac_features = [
    '✓ Very Sample Efficient',
    '✓ Great Exploration',
    '✓ Automatic Tuning (α)',
    '✓ Off-policy (reuse data)',
    '✗ Continuous only',
    '✗ More complex',
]
for i, feat in enumerate(sac_features):
    color = '#388e3c' if feat.startswith('✓') else '#d32f2f'
    ax.text(7, 7 - i*0.7, feat, fontsize=10, color=color)

# Use cases
ax.text(3, 1.3, 'Best for:', ha='center', fontsize=10, fontweight='bold')
ax.text(3, 0.7, 'Games, LLM alignment', ha='center', fontsize=10)

ax.text(9, 1.3, 'Best for:', ha='center', fontsize=10, fontweight='bold')
ax.text(9, 0.7, 'Robotics, control', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

---
## Summary: Key Takeaways

### Maximum Entropy RL

| Concept | Description |
|---------|-------------|
| **Objective** | Maximize reward + entropy |
| **Entropy** | H(π) = -E[log π(a\|s)] |
| **Temperature α** | Balances reward vs entropy |

### SAC Components

| Component | Purpose |
|-----------|--------|
| Twin Critics | Prevent Q overestimation |
| Replay Buffer | Sample efficiency |
| Auto Temperature | No manual tuning |
| Soft Updates | Stable training |

### PPO vs SAC

| Aspect | PPO | SAC |
|--------|-----|-----|
| **Policy Type** | On-policy | Off-policy |
| **Action Space** | Any | Continuous |
| **Sample Efficiency** | Lower | Higher |
| **Best For** | Games, RLHF | Robotics |

---
## Test Your Understanding

**1. What does "maximum entropy" mean in SAC?**
<details>
<summary>Click to reveal answer</summary>
SAC maximizes both expected reward AND policy entropy:
J(π) = E[Σ r(s,a)] + α × H(π)

High entropy means the policy maintains randomness/variety in its actions, which:
- Improves exploration
- Makes the policy more robust
- Keeps multiple good solutions ready
</details>

**2. Why does SAC use twin critics?**
<details>
<summary>Click to reveal answer</summary>
Twin critics (Q₁ and Q₂) help prevent overestimation of Q-values:
- Each critic may overestimate differently
- Taking min(Q₁, Q₂) gives a more conservative estimate
- This leads to more stable training

This technique comes from TD3 (Twin Delayed DDPG).
</details>

**3. Why is SAC more sample efficient than PPO?**
<details>
<summary>Click to reveal answer</summary>
SAC is off-policy, meaning it can reuse past experiences:
- Stores all transitions in a replay buffer
- Samples random batches for training
- Uses each experience many times

PPO is on-policy: uses data once then discards it.
</details>

**4. What does automatic temperature (α) mean?**
<details>
<summary>Click to reveal answer</summary>
SAC automatically adjusts α to maintain a target entropy:
- If policy becomes too deterministic → increase α (more exploration)
- If policy is too random → decrease α (more exploitation)

This removes the need for manual hyperparameter tuning!
</details>

**5. When should you choose SAC over PPO?**
<details>
<summary>Click to reveal answer</summary>
Choose SAC when:
- You have continuous actions (robotics, control)
- Data is expensive (real-world, slow simulations)
- Sample efficiency is critical
- You need good exploration

Choose PPO when:
- You have discrete actions (games, text)
- Simulations are fast/cheap
- You need maximum stability
- Doing RLHF for LLMs
</details>

---
## What's Next?

You've learned about SAC, the state-of-the-art for continuous control! In the next notebook, we'll compare all the advanced methods we've learned.

**Continue to:** [Notebook 5: Comparing Advanced Methods](05_comparing_advanced_methods.ipynb)

---

*SAC: "Stay random, stay creative, stay efficient!"*